<a href="https://colab.research.google.com/github/vkrisvasan/llamaKV/blob/main/llama31groqneo4jKV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
#https://console.groq.com/keys - register and get GROQ_API_KEY key
#https://console.groq.com/docs/libraries - Groq documentation
#https://docs.llamaindex.ai/en/stable/examples/cookbooks/GraphRAG_v1/
#https://github.com/projectwilsen/KnowledgeGraphLLM/blob/main/tutorial/2_convert_any_text_into_a_kg/notebook.ipynb
#https://docs.llamaindex.ai/en/stable/examples/cookbooks/llama3_cookbook_groq/
#https://www.youtube.com/watch?v=ky8LQE-82xs
#https://www.youtube.com/watch?v=KMXQ4SVLwmo

#https://docs.llamaindex.ai/en/stable/examples/property_graph/property_graph_neo4j/
#https://docs.llamaindex.ai/en/stable/understanding/loading/loading/
#https://console.neo4j.io/

!pip install groq
!pip install neo4j
!pip install llama-index
!pip install llama-index.core
!pip install llama-index-graph-stores-neo4j
!pip install llama-index-graph-stores-nebula
!pip install llama-index-graph-stores-neo4j-nebula

ERROR: Could not find a version that satisfies the requirement llama-index-graph-stores-neo4j-nebula (from versions: none)
ERROR: No matching distribution found for llama-index-graph-stores-neo4j-nebula


In [18]:
from groq import Groq
client = Groq()

In [19]:
!mkdir -p 'data/receipekv1'
!wget 'https://gutenberg.org/cache/epub/65061/pg65061.txt' -O 'data/receipekv1/receipekv11.txt'

--2024-08-04 01:13:10--  https://gutenberg.org/cache/epub/65061/pg65061.txt
Resolving gutenberg.org (gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to gutenberg.org (gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1398945 (1.3M) [text/plain]
Saving to: ‘data/receipekv1/receipekv11.txt’

data/receipekv1/rec 100%[===================>]   1.33M  5.35MB/s    in 0.2s    

2024-08-04 01:13:11 (5.35 MB/s) - ‘data/receipekv1/receipekv11.txt’ saved [1398945/1398945]



In [25]:

from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader("./data/receipekv1/").load_data()



In [26]:
from llama_index.core import PropertyGraphIndex

In [27]:
import os
import getpass
credential_names = ["GROQ_API_KEY","NEO4J_CONNECTION_URL","NEO4J_USER","NEO4J_PASSWORD"]
for credential in credential_names:
  if credential not in os.environ:
    os.environ[credential]=getpass.getpass("Provide your..." + credential)

In [28]:

from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

# Note: used to be `Neo4jPGStore`
graph_store = Neo4jPropertyGraphStore(
    username=os.environ["NEO4J_USER"],
    password=os.environ["NEO4J_PASSWORD"],
    url=os.environ["NEO4J_CONNECTION_URL"],
)

In [1]:
!pip install llama-index-embeddings-huggingface

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from llama_index.core import PropertyGraphIndex
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

index = PropertyGraphIndex.from_documents(
    documents,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    kg_extractors=[
        SchemaLLMPathExtractor(
            llm=OpenAI(model="gpt-3.5-turbo", temperature=0.0)
        )
    ],
    property_graph_store=graph_store,
    show_progress=True,
)

In [35]:

from llama_index.core import SchemaLLMPathExtractor

entities = Literal["PERSON", "PLACE", "THING"]
relations = Literal["PART_OF", "HAS", "IS_A"]
schema = {
    "PERSON": ["PART_OF", "HAS", "IS_A"],
    "PLACE": ["PART_OF", "HAS"],
    "THING": ["IS_A"],
}

kg_extractor = SchemaLLMPathExtractor(
  llm=llm,
  possible_entities=entities,
  possible_relations=relations,
  kg_validation_schema=schema,
  strict=True,  # if false, allows values outside of spec
)

ImportError: cannot import name 'SchemaLLMPathExtractor' from 'llama_index.core' (/usr/local/lib/python3.10/dist-packages/llama_index/core/__init__.py)

In [37]:
!pip install youtube-transcript-api

In [38]:
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader

loader = YoutubeTranscriptReader()
documents = loader.load_data(
    ytlinks=["https://www.youtube.com/watch?v=i3OYlaoj-BM"]
)

ModuleNotFoundError: No module named 'llama_index.readers.youtube_transcript'

In [ ]:
def generate_content(prompt):
  reponse = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt}
    ]
  )
  return reponse.choices[0].message.content

In [ ]:
output = generate_content("how to covert a pdf to a graph db")
print(output)

Converting a PDF to a Graph Database involves several steps:

1. **Text Extraction**: First, you need to extract the text from the PDF file. You can use tools like `pdfplumber`, `pdfminer`, or `Tesseract OCR` to extract the text.
2. **Tokenization**: Split the extracted text into individual words or tokens.
3. **Named Entity Recognition (NER)**: Identify named entities (e.g., names, locations, organizations) in the text.
4. **Relationship Extraction**: Identify relationships between entities in the text.
5. **Graph Construction**: Construct a graph using the extracted entities and relationships.

Here's a high-level example of how you can perform these steps using Python:

**Step 1: Text Extraction**

Use `pdfplumber` to extract the text from the PDF file:
```python
import pdfplumber

with pdfplumber.open("example.pdf") as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
```
**Step 2: Tokenization**

Split the extracted text into individual words or tok

In [ ]:
output = generate_content("with example can you explain SOLID design principle")
print(output)

The SOLID design principles!

**What is SOLID?**

SOLID is an acronym that stands for five design principles of object-oriented programming (OOP) that aim to promote maintainable, flexible, and scalable software design. These principles help developers create more robust, reusable, and easier-to-maintain code by:

1. Separating concerns
2. Reducing coupling
3. Increasing cohesion
4. Improving flexibility
5. Encouraging modularity

Here's a breakdown of each principle with examples:

### S - Single Responsibility Principle (SRP)

**One class should have one reason to change**

Meaning: A class should have a single responsibility or reason to change.

**Example:**

Bad code:
```python
class OrderProcessor:
    def process_order(self):
        # Process order
        pass

    def send_email(self):
        # Send email notification
        pass
```
Good code:
```python
class OrderProcessor:
    def process_order(self):
        # Process order
        pass

class EmailNotifier:
    def sen